In [ ]:
import argparse

import numpy as np
import torch
from fvcore.nn import FlopCountAnalysis, flop_count_table
from mmdet.apis import init_detector
from mmengine import DictAction
from mmyolo.utils import register_all_modules

register_all_modules()

In [ ]:
from os import listdir

config_root = '/home/ubuntu/mmyolo/configs/custom_dataset/'
for filename in listdir(config_root):
    if filename[-3:] != '.py':
        continue
    model_name = filename.split('_')[0] + '-' + filename.split('_')[1][:1]
    config = config_root + filename
    shape = [640, 640]

    if len(shape) == 1:
        h = w = shape[0]
    elif len(shape) == 2:
        h, w = shape
    else:
        raise ValueError('invalid input shape')
    # ori_shape = (1, 3, h, w)
    divisor = 32
    if divisor > 0:
        h = int(np.ceil(h / divisor)) * divisor
        w = int(np.ceil(w / divisor)) * divisor

    input_shape = (1, 3, h, w)

    # print('input shape is ', input_shape)

    model = init_detector(config, device='cuda:0')  # or device='cuda:0'

    try:
        model = init_detector(config, device='cpu')  # or device='cuda:0'
    except:
        print('Error', filename)

    flops = FlopCountAnalysis(model, torch.ones(input_shape))

    # params = parameter_count_table(model)
    flops_data = flop_count_table(flops)

    # FLOPS: floating point operations per second
    # FLOPs: floating point operations

    FLOP = flops_data.split('|')
    params, FLOPs = FLOP[10], FLOP[11]
    print(model_name, params, FLOPs)